In [1]:
from tabulate import tabulate
import import_ipynb
from Rule import Rule, Cell

importing Jupyter notebook from Rule.ipynb


In [2]:
#definisco la mia grammatica come se fosse un dizionario chiave valore 
class Dictlist(dict):
    def __setitem__(self, key, value):
        try:
            self[key]
            #print("Key:", self[key])
        except KeyError:
            super(Dictlist, self).__setitem__(key, [])
        self[key].append(value)

        #print("Key 2:", self[key].append(value))
        #print()

In [3]:
class Grammar(object):
    
    grammar_rules = Dictlist()
    parse_table = None
    length = 0
    tokens = []
    number_of_trees = 0
    
    def __init__(self, filename):
        self.grammar_rules = Dictlist()
        self.parse_table = None
        self.length = 0
        for line in open(filename):
            a, b = line.split("->")
            self.grammar_rules[b.rstrip().strip()]=a.rstrip().strip()

        #print("Grammar Rules: ", self.grammar_rules[b.strip().strip()]) #regole della grammatica in forma di dizionario : Grammar Rules:  {'NP VP': ['S'], 'X1 VP': ['S'], 'Aux NP': ['X1'] ecc..} dove la parte sinistra della regola di produzione è la chiave e la parte destra è il valore

    #stampo le regole della grammatica
    def print_rules(self):
        for r in self.grammar_rules: #parte destra della regola di produzione
            for p in self.grammar_rules[r]: #parte sinistra della regola di produzione
                print(str(p) + ' --> ' + str(r))
        

    #Applica le regole della grammatica ad un token. Questo metodo restituisce una lista di possibili regole di produzione per il token passato come parametro    
    def apply_rules(self,t):
        try:
            return self.grammar_rules[t]
        except KeyError as r:
            return None
            
    #Faccio il parsing della frase usando l'algoritmo CKY.
    def parse(self,sentence): 
        self.number_of_trees = 0 
        self.tokens = sentence.split()
        self.length = len(self.tokens)
        self.parse_table = [ [Cell() for x in range(self.length - y)] for y in range(self.length) ]
        
        #Process the first line. In questo ciclo for applico le regole della grammatica ad ogni token della frase.
        for x, t in enumerate(self.tokens): 
            r = self.apply_rules(t) #vedo le possibili regole per il token t
            if r == None:
                print("La parola " + str(t) + " non e presente nella grammatica.") #restituisce un errore se il token non è presente nella grammatica
            else:
                for w in r: #per ogni regola di produzione che posso applicare al token t
                    self.parse_table[0][x].add_production(w,Rule(t,None,None),None) #aggiungo la regola di produzione alla tabella di parsing nella posizione [0,x]

        
        #Run CYK-Parser. 
        for l in range(2,self.length+1): #parto con sottostringhe di lunghezza l=2 e arrivo fino alla lunghezza della frase
            for s in range(1,self.length-l+2): 
                for p in range(1,l-1+1): 
                    
                    t1 = self.parse_table[p-1][s-1].get_rules()  #primo token
                    t2 = self.parse_table[l-p-1][s+p-1].get_rules() #secondo token
                            
                    for a in t1: #per ogni regola di produzione nella cella per il token t1
                        for b in t2: #per ogni regola di produzione nella cella  per il token t2
                            r = self.apply_rules(str(a.get_type()) + " " + str(b.get_type())) #applico le regole di produzione per la coppia di token a e b
                    
                            #print("Rule: " + str(a.get_type) + " " + str(b.get_type) + " --> " + str(r)) 
                                    
                            if r is not None: #se la coppia di token a e b può essere prodotta da una regola di produzione
                                for w in r: #per ogni regola di produzione che posso applicare alla coppia di token a e b
                                    print('Regole Applicate : ' + str(w) + '[' + str(l) + ',' + str(s) + ']' + 
                                          ' --> ' + str(a.get_type()) + '[' + str(p) + ',' + str(s) + ']' + ' ' + str(b.get_type())+ '[' + str(l-p) + ',' + str(s+p) + ']') #stampa la regola di produzione applicata alla coppia di token a e b e la posizione nella tabella di parsing
                                    self.parse_table[l-1][s-1].add_production(w,a,b)

                                    #print()                        
                                    #print("Table:", (self.parse_table[l-1][s-1].get_types))

        self.number_of_trees = len(self.parse_table[self.length-1][0].get_types()) #numero di alberi che si possono generare
        if self.number_of_trees > 0: 
            print()
            print("------------- Stringa accettata dal linguaggio -------------")
            print("Numero di alberi che si possono generare: " + str(self.number_of_trees))
            print("------------------------------------------------------------")
            print()

        else:
            print("------------- Stringa non accettata dal linguaggio -------------")
            print("---------------------------------------------------------------")
    
                            
    #Print the CYK parse table for the given sentence.         
    def print_parse_table(self):
        lines = [] 
        
        for row in reversed(self.parse_table):
            l = []
            for cell in row:
                l.append(cell.get_types())
            lines.append(l)
        
        lines.append(self.tokens)
        print('')
        print(tabulate(lines))
        print('')